In [4]:
import pandas as pd
df = pd.read_csv('litho-Table 1.csv', low_memory = False)
df = df[df.columns[df.isnull().mean() < 0.25]]
df = df.drop([col for col in df.columns if 'FA' in col or 'INA' in col or 'AAS' in col], 
             axis = 1) #drop analytes that aren't ICP-MS analysis
df = df[['MASTERID', 'LAT', 'LONG',
         'STRAT'] + [col for col in df.columns if 'ICP' in col]]

# In this cell, I compile a list of the Canadian critical minerals by doing a basic web scrape. We'll compare this list
# to what we have in the dataframe from open.canada.ca.

# this webpage has a nice html table that I'll use, which is much easier to work with in Python. 
#upon visual comparison it's the same as the government list on https://www.nrcan.gc.ca/our-natural-resources/minerals-mining/critical-minerals/23414

#scrape the table of critical minerals
critical_minerals_df = pd.read_html('https://www.bennettjones.com/Blogs-Section/Canada-Announces-the-Critical-Minerals-List')[0]

#create a list of the critical minerals from the webpage
cm_list = []
for i in range(len(critical_minerals_df)):
    cm_list.extend(critical_minerals_df.values.tolist()[i])

# typed out each elements symbol, which we will append to a dictionary shortly:
element_abb = ['Al', 'Ga', 'Mo', 'Te', 'Sb', 'Ge', 'Ni', 'Sn', 'Bi',
              'C', 'Ni', 'Ti', 'Ce', 'He', 'Pt', 'W', 'Cr', 'In', 'K', 
              'U', 'Co', 'Li', 'Ree', 'V', 'Cu', 'Mg', 'Sc', 'Zn', 
              'F', 'Mn', 'Ta']

critical_minerals_dict = dict(zip(cm_list, element_abb))
    
#scrape rare earth elements to add to dictionary
ree_df = pd.read_html('https://en.wikipedia.org/wiki/Rare-earth_element')[1]
ree_df = ree_df[['Name', 'Symbol']]
ree_dict = ree_df.set_index('Name').to_dict()['Symbol']

#combine dictionaries
critical_minerals_dict = {**critical_minerals_dict, **ree_dict}

#create a list to compare to columns in our dataframe. If a column
#matches a critical mineral, we keep the column
cm_list = [v for i, v in critical_minerals_dict.items()]
critical_minerals_dict.pop('Graphite', None) #The chemical formula for Graphite (C) causes issues for this program, so I delete it here
# critical_minerals_dict
# cm_list

critical_minerals_list = []
for value in critical_minerals_dict.values():
    for column in df.columns:
        if value in column:
            if column not in critical_minerals_list:
                critical_minerals_list.append(column)
print('The following analytes in our dataset are critical minerals:\n',critical_minerals_list)

The following analytes in our dataset are critical minerals:
 ['Al_ICP_PCT', 'Ga_ICP_PPM', 'Mo_ICP_PPM', 'Te_ICP_PPM', 'Sb_ICP_PPM', 'Ni_ICP_PPM', 'Bi_ICP_PPM', 'Ti_ICP_PCT', 'W_ICP_PPM', 'Cr_ICP_PPM', 'K_ICP_PCT', 'U_ICP_PPM', 'Co_ICP_PPM', 'V_ICP_PPM', 'Cu_ICP_PPM', 'Mg_ICP_PCT', 'Sc_ICP_PPM', 'Zn_ICP_PPM', 'Fe_ICP_PCT', 'Mn_ICP_PPM', 'La_ICP_PPM']
